In [217]:
%config InlineBackend.figure_format = "retina"

In [218]:
%run "./pnl_script.ipynb"

In [219]:
import os
import sys
import re

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import talib
import pandas_ta as pta

import time
import datetime

In [220]:
class CDXData:
    def __init__(self, starttime, endtime, symbol, interval, exchange_code):
        self.start = starttime  # "DD-MM-YYYY
        self.end = endtime
        self.interval = interval
        self.symbol = symbol
        self.exchange_code = exchange_code

    def convert_to_millisecond(self, time):
        date, month, year = time.split(" ")[0].split("-")
        hours, minutes, seconds = time.split(" ")[1].split(":")
        date_time = datetime.datetime(int(year), int(
            month), int(date), int(hours), int(minutes))

        return datetime.datetime.timestamp(date_time)*1000

    def fetch_api_call(self, start, end):

        url = f"https://public.coindcx.com/market_data/candles?pair={self.exchange_code.upper()}-{self.symbol.upper()}&interval={self.interval}&startTime={start}&endTime={end}&limit=1000" # Replace 'SNTBTC' with the desired market pair.
        response = requests.get(url)
        data = response.json()
        return data

    def fetch_range_data(self):
        date, month, year = self.start.split("-")
        start_dt = datetime.date(int(year), int(month), int(date))
        date, month, year = self.end.split("-")
        end_dt = datetime.date(int(year), int(month), int(date))

        delta = datetime.timedelta(days=1)

        dates = []

        while start_dt <= end_dt:
            t1 = self.convert_to_millisecond(
                str(start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+" 00:00:00")
            t2 = self.convert_to_millisecond(
                str(start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+" 23:59:59")
            data = self.fetch_api_call(t1, t2)
            df = pd.DataFrame(data)

            while len(data) == 1000:
                t2 = df["time"].iloc[-1]
                data = self.fetch_api_call(t1, t2)
                df = pd.concat([df, pd.DataFrame(data)])

            df.to_csv("cdx_bin_btcusdt/CDX_"+self.symbol+"_"+str(start_dt.day) +
                      "-"+str(start_dt.month)+"-"+str(start_dt.year)+".csv", index=False)
            start_dt += delta
        print(dates)

    def combine_all_data(self):
        date, month, year = self.start.split("-")
        start_dt = datetime.date(int(year), int(month), int(date))
        date, month, year = self.end.split("-")
        end_dt = datetime.date(int(year), int(month), int(date))

        delta = datetime.timedelta(days=1)
        df = pd.DataFrame()
        while start_dt <= end_dt:
            try: 
                df_new = pd.read_csv("cdx_bin_btcusdt/CDX_"+self.symbol+"_"+str(
                start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+".csv")
                df = pd.concat([df, df_new], axis=0)
                
            except: 
                print("File not found")
                print("cdx_bin_btcusdt/CDX_"+self.symbol+"_"+str(
                start_dt.day)+"-"+str(start_dt.month)+"-"+str(start_dt.year)+".csv")
            
            start_dt += delta

        # sort df on basis of time ascending
        df = df.sort_values(by=['time'])
        df.to_csv("CDX_final.csv", index=False)
        return df

In [221]:
def get_all_data():
        return pd.read_csv("CDX_final.csv")

In [222]:
def get_sma(df):
    plt.figure(figsize=(20, 10))

    x = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    y = df["open"]
    
    window_size = 10
    # Calculate the simple moving average using the rolling() and mean() functions
    sma = df["open"].rolling(window=window_size).mean()
    df["sma_10"] = sma

In [223]:
def get_sma_cross(t1, t2, df):
    x = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    y = df["open"]

    plt.figure(figsize=(20, 10))
    
    sma_low = df["open"].rolling(window=t1).mean()
    sma_high = df["open"].rolling(window=t2).mean()
    df["sma_low"] = sma_low
    df["sma_high"] = sma_high

    # write to csv
    df.to_csv("CDX_sma.csv", index=False)

In [224]:
def get_bbands(df):
    x = df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    upper, middle, lower = talib.BBANDS(df["close"], timeperiod=20)
    bbands_talib = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                      "bb_ma": middle,
                                      "bb_high": upper}
                                )

    df["bb_low"] = bbands_talib["bb_low"]
    df["bb_ma"] = bbands_talib["bb_ma"]
    df["bb_high"] = bbands_talib["bb_high"]

In [225]:
def get_tick(ind, df): 
    return df.iloc[ind]

In [226]:
def apply_sma_cross_new(t1, t2, df):
    assert(len(df) > t2)
    temp = df.iloc[:t2]
    temp_df = pd.DataFrame(temp)
    
    # # create a dataframe with 2 columns sma_low and sma_high
    time = temp_df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    sma_low, sma_high = [], []
    
    sma_low = temp_df["open"].rolling(window=t1).mean()
    sma_high = temp_df["open"].rolling(window=t2).mean()
    
    sma_df = pd.DataFrame({'sma_low': sma_low, 'sma_high': sma_high})

    temp_df["sma_low"] = sma_low
    temp_df["sma_high"] = sma_high
    temp_df["cumulative_pnl"] = 0

    temp_df.reset_index(drop=True, inplace=True)

    count = 0

    for i in range(0, len(df)-t2):
        tick = get_tick(t2+i, df)


        sma_low_prev = sma_df.iloc[-1, 0]
        sma_high_prev = sma_df.iloc[-1, 1]


        sma_low_new = (sma_low_prev*t1 - temp_df.iloc[-t1, -9] + tick.iloc[-6])/t1
        sma_high_new = (sma_high_prev*t2 - temp_df.iloc[-t2, -9] + tick.iloc[-6])/t2


        new_row = pd.DataFrame({"sma_low": [sma_low_new], "sma_high": [sma_high_new]})
        sma_df = pd.concat([sma_df, new_row], ignore_index=True)



        avg_price = (tick["open"] + tick["close"])/2
        quantity_held = get_quantity_held()

        to_transact = 1

        if(sma_low_new > sma_high_new and sma_low_prev < sma_high_prev): #uptrend
            if(quantity_held < 0): 
                to_transact = abs(quantity_held) + 1
            record_transaction("buy", avg_price, to_transact, tick["time"])
            count += 1  
        if(sma_low_new < sma_high_new and sma_low_prev > sma_high_prev): #downtrend
            if(quantity_held > 0): 
                to_transact = abs(quantity_held) + 1
            record_transaction("sell", avg_price, to_transact, tick["time"])
            count += 1  
    
        cumulative_pnl = get_cumulative_cash() + get_quantity_held()*avg_price
        temp_df = pd.concat([temp_df, pd.DataFrame({'open': tick["open"], 'high': tick["high"], 'low': tick["low"], 'volume': tick["volume"],'close': tick["close"], 'time': tick["time"], 'sma_low': sma_low_new, 'sma_high': sma_high_new, 'cumulative_pnl': cumulative_pnl}, index=[t2+i])], axis=0)
        
    print(count, cumulative_pnl)
    return temp_df

In [227]:
def apply_sma_3cross_new(t1, t2, t3, df):
    assert(len(df) > t3)
    temp = df.iloc[:t3]
    temp_df = pd.DataFrame(temp)
    
    # # create a dataframe with 2 columns sma_low and sma_high
    time = temp_df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    sma_low, sma_med, sma_high = [], [], []
    
    sma_low = temp_df["open"].rolling(window=t1).mean()
    sma_med = temp_df["open"].rolling(window=t2).mean()
    sma_high = temp_df["open"].rolling(window=t3).mean()
    
    sma_df = pd.DataFrame({'sma_low': sma_low, 'sma_med': sma_med, 'sma_high': sma_high})

    temp_df["sma_low"] = sma_low
    temp_df["sma_med"] = sma_med
    temp_df["sma_high"] = sma_high
    temp_df["cumulative_pnl"] = 0

    temp_df.reset_index(drop=True, inplace=True)

    count = 0

    for i in range(0, len(df)-t3):
        tick = get_tick(t3+i, df)


        sma_low_prev = sma_df.iloc[-1, 0]
        sma_med_prev = sma_df.iloc[-1, 1]
        sma_high_prev = sma_df.iloc[-1, 2]


        sma_low_new = (sma_low_prev*t1 - temp_df.iloc[-t1, 0] + tick.iloc[0])/t1
        sma_med_new = (sma_med_prev*t2 - temp_df.iloc[-t2, 0] + tick.iloc[0])/t2
        sma_high_new = (sma_high_prev*t3 - temp_df.iloc[-t3, 0] + tick.iloc[0])/t3


        new_row = pd.DataFrame({"sma_low": [sma_low_new], "sma_med": [sma_med_new], "sma_high": [sma_high_new]})
        sma_df = pd.concat([sma_df, new_row], ignore_index=True)


        avg_price = (tick["open"] + tick["close"])/2
        quantity_held = get_quantity_held()


        to_transact = 0

        if(sma_low_new > sma_med_new and sma_low_prev < sma_med_prev): #uptrend
            if(quantity_held < 0):
                to_transact = abs(quantity_held)
                record_transaction("buy", avg_price, to_transact, tick["time"])
                count += 1  

        if(sma_low_new > sma_high_new and sma_low_prev < sma_high_prev): 
            to_transact += 1
            record_transaction("buy", avg_price, to_transact, tick["time"])
            count += 1  
    
        if (sma_low_new < sma_med_new and sma_low_prev > sma_med_prev): #downtrend
            if(quantity_held > 0): 
                to_transact = quantity_held
                record_transaction("sell", avg_price, to_transact, tick["time"])
                count += 1 

        if(sma_low_new < sma_high_new and sma_low_prev > sma_high_prev): 
            to_transact = abs(quantity_held) + 1
            record_transaction("sell", avg_price, to_transact, tick["time"])
            count += 1  
            
    
        cumulative_pnl = get_cumulative_cash() + get_quantity_held()*avg_price + 20000
        temp_df = pd.concat([temp_df, pd.DataFrame({'open': tick["open"], 'high': tick["high"], 'low': tick["low"], 'volume': tick["volume"],'close': tick["close"], 'time': tick["time"], 'sma_low': sma_low_new, 'sma_med': sma_med_new, 'sma_high': sma_high_new, 'cumulative_pnl': cumulative_pnl}, index=[t2+i])], axis=0)
        
    print(count, cumulative_pnl)
    return temp_df

In [228]:
obj = CDXData("1-08-2022", "30-05-2023", "BTC_USDT", "1h", "B")
# obj.fetch_range_data()
# obj.combine_all_data()
with open('pnl/sma.csv', 'w') as f:
        f.write('typeOfTransaction,price,quantity,timeOfTransaction,timestamp,quantity_held,pnl,cash_balance,portfolio_value\n')

df = get_all_data()
temp_df = apply_sma_3cross_new(100, 140, 200, df)
# temp_df = apply_sma_cross_new(100, 200, df)
# df = get_sma_cross(20, 40, df)
# get_bbands(df)

time = temp_df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
price, sma_low, sma_high, sma_med, cumulative_pnl = temp_df["open"], temp_df["sma_low"], temp_df["sma_high"], temp_df["sma_med"], temp_df["cumulative_pnl"]

plt.figure(figsize=(90, 35))

plt.plot(time, price, label='Price', color='blue')
plt.plot(time, sma_low, label='SMA_Low', color='orange')
plt.plot(time, sma_med, label='SMA_Med', color='red')
plt.plot(time, sma_high, label='SMA_High', color='green')
plt.plot(time, cumulative_pnl, label='Cumulative PnL', color='black')

plt.legend(loc='upper left')

temp_df

# plt.plot(time, bb_low, color="red")
# plt.plot(time, bb_ma, color="blue")
# plt.plot(time, bb_high, color="red")

57 23786.975


,open,high,low,volume,close,time,sma_low,sma_med,sma_high,cumulative_pnl
0,23748.65,23982.89,23700.37,6209.96200,23845.88,1.659294e+12,NaN,NaN,NaN,0.000
1,23843.69,23892.43,23772.75,3908.90950,23797.15,1.659298e+12,NaN,NaN,NaN,0.000
2,23797.22,23827.25,23531.61,3905.49020,23589.58,1.659301e+12,NaN,NaN,NaN,0.000
3,23591.06,23593.45,23227.31,9707.91070,23377.48,1.659305e+12,NaN,NaN,NaN,0.000
4,23379.82,23469.00,23266.60,5601.01250,23293.32,1.659308e+12,NaN,NaN,NaN,0.000
...,...,...,...,...,...,...,...,...,...,...
6471,27800.02,27865.00,27621.00,235612.55732,27622.65,1.685455e+12,27283.6282,27013.771286,26998.04750,23767.330
6472,27622.65,27738.01,27615.00,158824.11716,27673.08,1.685459e+12,27295.3535,27023.786286,27002.05235,23703.860
6473,27673.08,27743.48,27554.00,194690.64247,27713.43,1.685462e+12,27307.4514,27033.934429,27006.20215,23749.250
6474,27713.43,27776.15,27625.14,109951.94393,27696.12,1.685466e+12,27320.1853,27043.286214,27010.63620,23760.770
